In [5]:
import time
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

import warnings
warnings.filterwarnings('ignore')

import gc

In [6]:
df_train = pd.read_csv('train.csv', engine='python', encoding='utf-8', error_bad_lines=False)


Skipping line 4322: unexpected end of data


In [7]:
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [8]:
print("Size of the training dataset is", df_train.shape)


Size of the training dataset is (4320, 6)


In [9]:
df_train.dtypes

PRODUCT_ID           int64
TITLE               object
BULLET_POINTS       object
DESCRIPTION         object
PRODUCT_TYPE_ID      int64
PRODUCT_LENGTH     float64
dtype: object

In [10]:
df_train.astype('object').describe().transpose()

,count,unique,top,freq
PRODUCT_ID,4320,4320,1925202,1
TITLE,4320,4319,Honey,2
BULLET_POINTS,2697,2599,"[Good quality and Suitable to use.,This Produc...",23
DESCRIPTION,2065,2009,This Case is Made up of Hard Polycarbonate Pla...,20
PRODUCT_TYPE_ID,4320,1442,1,219
PRODUCT_LENGTH,4320.0,912.0,600.0,249.0


In [11]:
df_train.isna().sum()

PRODUCT_ID            0
TITLE                 0
BULLET_POINTS      1623
DESCRIPTION        2255
PRODUCT_TYPE_ID       0
PRODUCT_LENGTH        0
dtype: int64

In [12]:
df_train=df_train.dropna()

In [13]:
df_train.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

In [16]:
lb = LabelBinarizer(sparse_output=True)
tv = TfidfVectorizer(max_features=55000, ngram_range=(1, 2), stop_words='english')

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:
stop = stopwords.words('english')
lem = WordNetLemmatizer()

<b> BULLET POINTS </b>

In [19]:
def cleanText(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(cleanText)

In [20]:
def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [21]:
def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(remove_stopwords)
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(word_lem)

In [22]:
df_train.head(10)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,loud dual tone trumpet horn compatible sx elec...,"Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,made cotton lycra give way stretchable comfort...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,simple elegant great displaying indoor plant l...,HINS Brings you the most Elegant Looking Pot w...,5725,950.000000
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,high quality pvc material kitchen aluminum foi...,<p><strong>Aluminum Foil Stickers-good kitchen...,6030,984.251967
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,pure natural essential oil fragrance oil bottl...,"Transform your home, workplace or hotel room i...",8201,393.700787
10,2857066,3NH® Glasses Goggles Anti Fog Antis Windproof ...,good quality suitable use product come proper ...,3NH Glasses Goggles Anti Fog Antis Windproof A...,10359,590.551180
14,2790448,SEGOVIA Single Walled Stainless Steel Sports| ...,segovia bottle consists stainless steel provid...,Segovia bottle consists of stainless steel whi...,1273,314.960630
21,2964715,Twisted Swirl Vintage Blue Phone Case Compatib...,compatible mode mobile phone case compatible i...,<b>Welcome to our store. We aim to offer uniqu...,12556,577.000000
25,2120903,JSR CART Incense Cones Rose (Pack of 65 Cones),completed natural herbal fragrance rose order ...,PACK OF 65 INCENSE CONES. Best Quality Low Smo...,1582,393.700787
27,2273102,Redgem 925 Silver Stud Earrings for Girls and ...,gemstone amethyst amethyst well known mineral ...,<p>These Stylish Earring for Women and Girls a...,3357,31.496000


In [23]:
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].astype(str)
print(df_train['BULLET_POINTS'].apply(lambda x: len(x.split(' '))).sum())

116474


In [24]:
import string
# Create a function to remove punctuations
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [25]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(porter.stem)
df_train['BULLET_POINTS'] = df_train['BULLET_POINTS'].apply(remove_punctuation)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# CountVectorizer - name & categories
cv = CountVectorizer(min_df=10)
X_name = cv.fit_transform(df_train['BULLET_POINTS'])

In [27]:
from collections import Counter
def reducecolumns(Col):
    # remove columns with <= 1% non-zero entries (arbitrary choice so that notebook doesn't crash)
    n_docs = Counter(Col.nonzero()[1])
    cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * Col.shape[0]]
    return Col[:, cols_to_keep]

In [28]:
print("Item Name Shape: " + str(reducecolumns(X_name).shape))

Item Name Shape: (1962, 1651)


In [29]:
X_brand1 = lb.fit_transform(df_train['BULLET_POINTS'])
X_description1 = tv.fit_transform(df_train['BULLET_POINTS'])

In [30]:
tfidf_weights = dict(zip(tv.get_feature_names_out(), tv.idf_))
tfidf_weights = pd.DataFrame(columns = ['BULLET_POINTS']).from_dict(dict(tfidf_weights), orient = 'index')
tfidf_weights.columns = ['BULLET_POINTS']

#Lowest tfidf scores -  Highest frequency words that get most importance:
tfidf_weights.sort_values(by=['BULLET_POINTS'], ascending=True).head(10)

,BULLET_POINTS
material,2.188638
quality,2.203803
easy,2.268681
design,2.309352
size,2.371629
high,2.393965
use,2.504587
color,2.530140
product,2.553951
perfect,2.556363


<b> DESCRIPTION </b>

In [31]:
def cleanText(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(cleanText)

In [32]:
def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [33]:
def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(remove_stopwords)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(word_lem)

In [34]:
df_train.head(10)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,loud dual tone trumpet horn compatible sx elec...,specification color red material aluminium vol...,7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,made cotton lycra give way stretchable comfort...,aishah woman lycra cotton ankel legging brand ...,2996,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,simple elegant great displaying indoor plant l...,hin brings elegant looking pot stand durable l...,5725,950.000000
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,high quality pvc material kitchen aluminum foi...,p strong aluminum foil sticker good kitchen he...,6030,984.251967
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,pure natural essential oil fragrance oil bottl...,transform home workplace hotel room personal a...,8201,393.700787
10,2857066,3NH® Glasses Goggles Anti Fog Antis Windproof ...,good quality suitable use product come proper ...,nh glass goggles anti fog anti windproof anti ...,10359,590.551180
14,2790448,SEGOVIA Single Walled Stainless Steel Sports| ...,segovia bottle consists stainless steel provid...,segovia bottle consists stainless steel provid...,1273,314.960630
21,2964715,Twisted Swirl Vintage Blue Phone Case Compatib...,compatible mode mobile phone case compatible i...,b welcome store aim offer unique special case ...,12556,577.000000
25,2120903,JSR CART Incense Cones Rose (Pack of 65 Cones),completed natural herbal fragrance rose order ...,pack incense cone best quality low smoke dhoop...,1582,393.700787
27,2273102,Redgem 925 Silver Stud Earrings for Girls and ...,gemstone amethyst amethyst well known mineral ...,p stylish earring woman girl handcrafted love ...,3357,31.496000


In [35]:
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].astype(str)
print(df_train['DESCRIPTION'].apply(lambda x: len(x.split(' '))).sum())

142729


In [36]:
import string
# Create a function to remove punctuations
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

In [37]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
df_train['DECSRIPTION'] = df_train['DESCRIPTION'].apply(porter.stem)
df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(remove_punctuation)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# CountVectorizer - name & categories
cv = CountVectorizer(min_df=10)
X_name2 = cv.fit_transform(df_train['DESCRIPTION'])

In [39]:
from collections import Counter
def reducecolumns(Col):
    # remove columns with <= 1% non-zero entries (arbitrary choice so that notebook doesn't crash)
    n_docs = Counter(Col.nonzero()[1])
    cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * Col.shape[0]]
    return Col[:, cols_to_keep]

In [40]:
print("Item Name Shape: " + str(reducecolumns(X_name2).shape))

Item Name Shape: (1962, 1786)


In [41]:
X_brand2 = lb.fit_transform(df_train['DESCRIPTION'])
X_description2 = tv.fit_transform(df_train['DESCRIPTION'])

In [42]:
tfidf_weights = dict(zip(tv.get_feature_names_out(), tv.idf_))
tfidf_weights = pd.DataFrame(columns = ['DESCRIPTION']).from_dict(dict(tfidf_weights), orient = 'index')
tfidf_weights.columns = ['DESCRIPTION']

#Lowest tfidf scores -  Highest frequency words that get most importance:
tfidf_weights.sort_values(by=['DESCRIPTION'], ascending=True).head(10)

,DESCRIPTION
quality,2.322648
design,2.359653
color,2.393965
material,2.408443
size,2.472982
product,2.530140
br,2.532496
easy,2.558782
high,2.588268
make,2.628986


<b> TITLE </b>

In [43]:
def cleanText(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ",words)
    text = words.lower().split()
    return " ".join(text)

df_train['TITLE'] = df_train['TITLE'].apply(cleanText)

In [44]:
def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

In [45]:
def word_lem(text):
    """The function to apply lemmatizing"""
    lem_text = [lem.lemmatize(word) for word in text.split()]
    return " ".join(lem_text)
df_train['TITLE'] = df_train['TITLE'].apply(remove_stopwords)
df_train['TITLE'] = df_train['TITLE'].apply(word_lem)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# CountVectorizer - name & categories
cv = CountVectorizer(min_df=10)
X_name3 = cv.fit_transform(df_train['TITLE'])

In [47]:
from collections import Counter
def reducecolumns(Col):
    # remove columns with <= 1% non-zero entries (arbitrary choice so that notebook doesn't crash)
    n_docs = Counter(Col.nonzero()[1])
    cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * Col.shape[0]]
    return Col[:, cols_to_keep]

In [48]:
print("Item Name Shape: " + str(reducecolumns(X_name3).shape))

Item Name Shape: (1962, 491)


In [49]:
X_brand3 = lb.fit_transform(df_train['TITLE'])
X_description3 = tv.fit_transform(df_train['TITLE'])

In [50]:
tfidf_weights = dict(zip(tv.get_feature_names_out(), tv.idf_))
tfidf_weights = pd.DataFrame(columns = ['TITLE']).from_dict(dict(tfidf_weights), orient = 'index')
tfidf_weights.columns = ['TITLE']

#Lowest tfidf scores -  Highest frequency words that get most importance:
tfidf_weights.sort_values(by=['TITLE'], ascending=True).head(10)

,TITLE
woman,2.765118
cover,2.871802
black,3.021548
pack,3.378223
case,3.383732
inch,3.482363
printed,3.482363
set,3.545277
men,3.564949
girl,3.612416


In [51]:
X_dummies = csr_matrix(pd.get_dummies(df_train[['TITLE', 'BULLET_POINTS', 'DESCRIPTION']], sparse=True).values)

In [52]:
from scipy.sparse import csr_matrix, hstack

In [53]:
# Combine everything together using Sparse matrix
sparse_merge = hstack((X_dummies, reducecolumns(X_brand1), reducecolumns(X_description1), reducecolumns(X_brand2), reducecolumns(X_description2), reducecolumns(X_brand3), reducecolumns(X_description3))).tocsr()

print(sparse_merge.shape)

(1962, 147857)


In [54]:
target = np.log(df_train['PRODUCT_LENGTH']+1)

print(target.shape)

(1962,)


In [55]:
from collections import Counter

n_docs = Counter(sparse_merge.nonzero()[1])
cols_to_keep = [k for k, v in n_docs.items() if v > .0005 * sparse_merge.shape[0]]
sparse_merge_filtered = sparse_merge[:, cols_to_keep]

In [56]:
#Split into train and test sets
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(sparse_merge, target, test_size = 0.35, random_state = 1)

In [57]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [60]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.2 MB/s eta 0:00:00


In [62]:
from sklearn.pipeline import Pipeline

In [66]:
def regressor_model():  
    model = Sequential()
    model.add(Dense(units=7, kernel_initializer='uniform', activation='relu', input_dim=147857))
    model.add(Dense(1, kernel_initializer='uniform'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    return model

features_train_processed = features_train.toarray()  
estimators = []
estimators.append(('mlp', KerasRegressor(build_fn=regressor_model, verbose=1)))
pipeline = Pipeline(estimators)
pipeline.fit(features_train_processed, target_train)

40/40 [==============================] - 3s 55ms/step - loss: 43.2745 - mae: 6.4869


Pipeline(steps=[('mlp',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fc575ccb2e0>)])

In [67]:
target_pred = pipeline.predict(features_test.toarray())

22/22 [==============================] - 1s 23ms/step


In [69]:
target_pred_inv=np.expm1(target_pred)

In [70]:
target_pred_inv

array([0.12258972, 0.11502382, 0.134829  , 0.12886243, 0.1799021 ,
       0.10907458, 0.15045531, 0.16572763, 0.0856473 , 0.18040182,
       0.10747891, 0.14644107, 0.15200794, 0.07268979, 0.15216753,
       0.10857835, 0.11633762, 0.07318657, 0.14683111, 0.13875401,
       0.18698056, 0.17511252, 0.1357872 , 0.17548415, 0.16907498,
       0.12102672, 0.07899794, 0.12060565, 0.10947713, 0.08207513,
       0.1247391 , 0.1340715 , 0.12298714, 0.13692413, 0.12834936,
       0.14297636, 0.16211344, 0.14685458, 0.11070095, 0.1454859 ,
       0.1435513 , 0.10187039, 0.10725415, 0.1230858 , 0.10799735,
       0.09201147, 0.15472221, 0.13737664, 0.08063167, 0.0855483 ,
       0.17711699, 0.17415942, 0.18250139, 0.09681537, 0.11984471,
       0.1232046 , 0.09386039, 0.13457213, 0.08763991, 0.19866751,
       0.0891853 , 0.19066891, 0.13094255, 0.10105555, 0.14715698,
       0.10490736, 0.14246325, 0.14422439, 0.15089728, 0.1707308 ,
       0.16562235, 0.11568518, 0.17314346, 0.13865979, 0.18637

In [71]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [72]:
rmse = np.sqrt(mean_squared_error(target_test, target_pred_inv))

print("RMSE:", rmse)

RMSE: 6.599393227371408


In [73]:
smape = np.mean(2.0 * np.abs(target_pred_inv - target_test) / (np.abs(target_pred_inv) + np.abs(target_test))) * 100.0

print("SMAPE:", smape)

SMAPE: 191.90912517452855
